In [ ]:
!git clone -b master https://github.com/charles9n/bert-sklearn
!cd bert-sklearn; pip install .
import os
os.chdir("bert-sklearn")
print(os.listdir())

Cloning into 'bert-sklearn'...
remote: Enumerating objects: 259, done.
remote: Total 259 (delta 0), reused 0 (delta 0), pack-reused 259
Receiving objects: 100% (259/259), 516.15 KiB | 4.82 MiB/s, done.
Resolving deltas: 100% (131/131), done.
Processing /content/bert-sklearn
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 827.4 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 39.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 53.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 37.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import re
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
import pandas as pd
import json
import nltk
from bert_sklearn import BertTokenClassifier
from bert_sklearn import load_model
nltk.download('punkt')
nltk.download('stopwords')
import os
import math
import random
import csv
import sys

import numpy as np
from sklearn import metrics
from sklearn.metrics import f1_score, precision_score, recall_score
from sklearn.metrics import classification_report
import statistics as stats


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
def read_data(filepath):
    # 打开文本文件并逐行读取
    with open(filepath, 'r') as file:
        lines = file.readlines()

    # 初始化两个空列表，用于存储解析后的 "text" 和 "label" 数据
    text_list = []
    label_list = []

    # 逐行解析 JSON 数据并提取 "text" 和 "label" 字段
    for line in lines:
        # 使用 json.loads 解析每一行的 JSON 数据
        parsed_line = json.loads(line)
        # 提取 "text" 和 "label" 字段并添加到相应列表中
        text_list.append(parsed_line['text'])
        label_list.append(parsed_line['label'])

    # 创建 Pandas DataFrame
    df = pd.DataFrame({'text': text_list, 'label': label_list})
    return df

DATADIR='/content/drive/MyDrive/'
train_path='train.txt'
test_path='test.txt'
dev_path='dev.txt'

train=read_data(DATADIR+train_path)
test=read_data(DATADIR+test_path)
dev=read_data(DATADIR+dev_path)
# 打印 Pandas DataFrame
print(train)


                                                   text       label
0     However, how frataxin interacts with the Fe-S ...  background
1     In the study by Hickey et al. (2012), spikes w...  background
2     The drug also reduces catecholamine secretion,...  background
3     By clustering with lowly aggressive close kin ...  background
4     Ophthalmic symptoms are rare manifestations of...  background
...                                                 ...         ...
7315  cells in the lungs and slow the production of ...  background
7316  latter phenotypes are called competitive adapt...  background
7317  While we cannot exclude alternative models, ou...      result
7318  Generation of macrophages from patient-derived...  background
7319  The same threshold pH has been measured for SF...  background

[7320 rows x 2 columns]


In [ ]:
def clean_text(text):
    # 去除特殊字符和标点符号
    text = re.sub(r'[^\w\s]', '', text)
    # 去除数字
    text = re.sub(r'\d+', '', text)
    return text

def tokenize_text(text):
    tokens = word_tokenize(text)
    return tokens

def remove_stopwords(tokens):
    stop_words = set(stopwords.words('english'))
    filtered_tokens = [word for word in tokens if word.lower() not in stop_words]
    return filtered_tokens

def stem_text(tokens):
    stemmer = PorterStemmer()
    stemmed_tokens = [stemmer.stem(word) for word in tokens]
    return stemmed_tokens

def lemmatize_text(tokens):
    lemmatizer = WordNetLemmatizer()
    lemmatized_tokens = [lemmatizer.lemmatize(word) for word in tokens]
    return lemmatized_tokens

def preprocess_text(text):
    cleaned_text = clean_text(text)
    tokens = tokenize_text(cleaned_text)
    filtered_tokens = remove_stopwords(tokens)
    stemmed_tokens = stem_text(filtered_tokens)
    return stemmed_tokens

# def preprocess_data(df):
#     df['cleaned_text'] = df['text'].apply(clean_text)
#     df['tokens'] = df['cleaned_text'].apply(tokenize_text)
#     df['filtered_tokens'] = df['tokens'].apply(remove_stopwords)
#     #df['stemmed_tokens'] = df['filtered_tokens'].apply(stem_text)
#     df=df.drop(columns=['text','cleaned_text','tokens'])
#     return df

# # 示例用法
# train_processed = preprocess_data(train)
# test_processed = preprocess_data(test)
# dev_processed = preprocess_data(dev)
# train_processed.head()

In [ ]:
# 不做处理的data
def preprocess_data(df):
    df['cleaned_text'] = df['text'].apply(clean_text)
    df['tokens'] = df['cleaned_text'].apply(tokenize_text)
    df['filtered_tokens'] = df['tokens'].apply(remove_stopwords)
    df['stemmed_tokens'] = df['filtered_tokens'].apply(stem_text)
    #df=df.drop(columns=['cleaned_text','tokens','filtered_tokens'])
    return df
# 示例用法
train_processed = preprocess_data(train)
test_processed = preprocess_data(test)
dev_processed = preprocess_data(dev)
train_processed.head()

,text,label,cleaned_text,tokens,filtered_tokens,stemmed_tokens
0,"However, how frataxin interacts with the Fe-S ...",background,However how frataxin interacts with the FeS cl...,"[However, how, frataxin, interacts, with, the,...","[However, frataxin, interacts, FeS, cluster, b...","[howev, frataxin, interact, fe, cluster, biosy..."
1,"In the study by Hickey et al. (2012), spikes w...",background,In the study by Hickey et al spikes were samp...,"[In, the, study, by, Hickey, et, al, spikes, w...","[study, Hickey, et, al, spikes, sampled, field...","[studi, hickey, et, al, spike, sampl, field, p..."
2,"The drug also reduces catecholamine secretion,...",background,The drug also reduces catecholamine secretion ...,"[The, drug, also, reduces, catecholamine, secr...","[drug, also, reduces, catecholamine, secretion...","[drug, also, reduc, catecholamin, secret, ther..."
3,By clustering with lowly aggressive close kin ...,background,By clustering with lowly aggressive close kin ...,"[By, clustering, with, lowly, aggressive, clos...","[clustering, lowly, aggressive, close, kin, Ki...","[cluster, lowli, aggress, close, kin, king, ab..."
4,Ophthalmic symptoms are rare manifestations of...,background,Ophthalmic symptoms are rare manifestations of...,"[Ophthalmic, symptoms, are, rare, manifestatio...","[Ophthalmic, symptoms, rare, manifestations, i...","[ophthalm, symptom, rare, manifest, intracrani..."


In [ ]:
#text

X_train=train_processed['text']
y_train=train_processed['label']
X_dev=dev_processed['text']
y_dev=dev_processed['label']
X_test=test_processed['text']
y_test=test_processed['label']

# 从 DataFrame 中获取 'label' 列作为标签列表
#label_list = train_processed['label'].unique()
label_list=set(y_train)
# 打印标签列表
print(label_list)

{'result', 'method', 'background'}


In [ ]:
#'cleaned_text

X_train=train_processed['cleaned_text']
y_train=train_processed['label']
X_dev=dev_processed['cleaned_text']
y_dev=dev_processed['label']
X_test=test_processed['cleaned_text']
y_test=test_processed['label']

# 从 DataFrame 中获取 'label' 列作为标签列表
#label_list = train_processed['label'].unique()
label_list=set(y_train)
# 打印标签列表
print(label_list)

{'result', 'method', 'background'}


In [ ]:
#token

X_train=train_processed['tokens']
y_train=train_processed['label']
X_dev=dev_processed['tokens']
y_dev=dev_processed['label']
X_test=test_processed['tokens']
y_test=test_processed['label']

# 从 DataFrame 中获取 'label' 列作为标签列表
#label_list = train_processed['label'].unique()
label_list=set(y_train)
# 打印标签列表
print(label_list)

{'result', 'method', 'background'}


In [ ]:
#'filtered_tokens'
X_train=train_processed['filtered_tokens']
y_train=train_processed['label']
X_dev=dev_processed['filtered_tokens']
y_dev=dev_processed['label']
X_test=test_processed['filtered_tokens']
y_test=test_processed['label']

# 从 DataFrame 中获取 'label' 列作为标签列表
#label_list = train_processed['label'].unique()
label_list=set(y_train)
# 打印标签列表
print(label_list)

{'result', 'method', 'background'}


In [ ]:
X_train

0       [However, frataxin, interacts, FeS, cluster, b...
1       [study, Hickey, et, al, spikes, sampled, field...
2       [drug, also, reduces, catecholamine, secretion...
3       [clustering, lowly, aggressive, close, kin, Ki...
4       [Ophthalmic, symptoms, rare, manifestations, i...
                              ...                        
7315    [cells, lungs, slow, production, cytokines, pr...
7316    [latter, phenotypes, called, competitive, adap...
7317    [exclude, alternative, models, findings, consi...
7318    [Generation, macrophages, patientderived, iPSC...
7319    [threshold, pH, measured, SFVinduced, cell, fu...
Name: filtered_tokens, Length: 7320, dtype: object

In [ ]:
# 检查标签列表和映射
print("标签列表:", label_list)
print("标签到ID的映射:", model.label2id)

# 调试训练数据中的标签
print("y_train中的唯一标签:", set(y_train))

标签列表: {'method', 'background', 'result'}
标签到ID的映射: {'method': 0, 'background': 1, 'result': 2}
y_train中的唯一标签: {'method', 'background', 'result'}


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

# 创建TF-IDF向量化器
vectorizer = TfidfVectorizer(max_features=10000)

# 将文本数据转换为TF-IDF特征向量
X_train = vectorizer.fit_transform(X_train_str)
X_train

<7320x10000 sparse matrix of type '<class 'numpy.float64'>'
	with 109955 stored elements in Compressed Sparse Row format>

In [ ]:
# filter-token
model = BertTokenClassifier('bert-base-uncased',
                            max_seq_length=178,
                            epochs=3,
                            gradient_accumulation_steps=4,
                            learning_rate=3e-5,
                            train_batch_size=32,
                            eval_batch_size=32)
                            #label_list=label_list)

print(model)

# finetune model on train data
model.fit(X_train, y_train)



Building sklearn token classifier...
BertTokenClassifier(eval_batch_size=32, gradient_accumulation_steps=4,
                    learning_rate=3e-05, max_seq_length=178)


100%|██████████| 231508/231508 [00:00<00:00, 1245307.64B/s]


Loading bert-base-uncased model...


100%|██████████| 433/433 [00:00<00:00, 209787.87B/s]


Defaulting to linear classifier/regressor
Loading Pytorch checkpoint

train data size: 6588, validation data size: 732



...


Epoch 2, Train loss: 0.0179, Val loss: 0.0160, Val accy: 86.60%, f1: 86.60



Training  :   0%|          | 0/824 [00:00<?, ?it/s]

Validating:   0%|          | 0/23 [00:00<?, ?it/s]


Epoch 3, Train loss: 0.0110, Val loss: 0.0175, Val accy: 87.03%, f1: 87.03



BertTokenClassifier(do_lower_case=True, eval_batch_size=32,
                    gradient_accumulation_steps=4,
                    label_list=array(['a', 'b', 'c', 'd', 'e', 'g', 'h', 'k', 'l', 'm', 'n', 'o', 'r',
       's', 't', 'u'], dtype='<U1'),
                    learning_rate=3e-05, max_seq_length=178)

In [ ]:
#filter——token
# 导入所需的库
from sklearn.metrics import classification_report

# 获取模型在测试集上的预测结果
y_preds = model2.predict(X_test)

# 打印模型在测试集上的分类报告
print(classification_report(y_test, y_preds))


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


Predicting:   0%|          | 0/59 [00:00<?, ?it/s]

              precision    recall  f1-score   support

  background       0.85      0.83      0.84       997
      method       0.84      0.79      0.81       605
      result       0.66      0.82      0.73       259

    accuracy                           0.82      1861
   macro avg       0.78      0.81      0.80      1861
weighted avg       0.82      0.82      0.82      1861



In [ ]:
#model.score(X_test, y_test)
savefile='base_uncased_model2.bin'
model2.save(savefile)

In [ ]:
#filter_token
from bert_sklearn import BertClassifier
model2 = BertClassifier('bert-base-uncased',
                            max_seq_length=178,
                            epochs=3,
                            gradient_accumulation_steps=4,
                            learning_rate=3e-5,
                            train_batch_size=32,
                            eval_batch_size=32)
                            #label_list=label_list)

print(model2)

# finetune model on train data
model2.fit(X_train, y_train)



Building sklearn text classifier...
BertClassifier(eval_batch_size=32, gradient_accumulation_steps=4,
               learning_rate=3e-05, max_seq_length=178)
Loading bert-base-uncased model...
Defaulting to linear classifier/regressor
Loading Pytorch checkpoint

train data size: 6588, validation data size: 732



Training  :   0%|          | 0/824 [00:00<?, ?it/s]

Validating:   0%|          | 0/23 [00:00<?, ?it/s]


Epoch 1, Train loss: 0.6620, Val loss: 0.6125, Val accy: 74.73%



Training  :   0%|          | 0/824 [00:00<?, ?it/s]

...


Epoch 2, Train loss: 0.3811, Val loss: 0.4726, Val accy: 80.87%



Training  :   0%|          | 0/824 [00:00<?, ?it/s]

Validating:   0%|          | 0/23 [00:00<?, ?it/s]


Epoch 3, Train loss: 0.2344, Val loss: 0.5122, Val accy: 80.60%



BertClassifier(do_lower_case=True, eval_batch_size=32,
               gradient_accumulation_steps=4,
               label_list=array(['background', 'method', 'result'], dtype=object),
               learning_rate=3e-05, max_seq_length=178)

In [ ]:
# 原始text
from bert_sklearn import BertClassifier
model2 = BertClassifier('bert-base-uncased',
                            max_seq_length=178,
                            epochs=3,
                            gradient_accumulation_steps=4,
                            learning_rate=3e-5,
                            train_batch_size=32,
                            eval_batch_size=32)
                            #label_list=label_list)

print(model2)

# finetune model on train data
model2.fit(X_train, y_train)

# 导入所需的库
from sklearn.metrics import classification_report

# 获取模型在测试集上的预测结果
y_preds = model2.predict(X_test)

# 打印模型在测试集上的分类报告
print(classification_report(y_test, y_preds))

Building sklearn text classifier...
BertClassifier(eval_batch_size=32, gradient_accumulation_steps=4,
               learning_rate=3e-05, max_seq_length=178)


100%|██████████| 231508/231508 [00:00<00:00, 2720981.59B/s]


Loading bert-base-uncased model...


100%|██████████| 433/433 [00:00<00:00, 516240.37B/s]


Defaulting to linear classifier/regressor
Loading Pytorch checkpoint

train data size: 6588, validation data size: 732



...

Training  :   0%|          | 0/824 [00:00<?, ?it/s]

Validating:   0%|          | 0/23 [00:00<?, ?it/s]


Epoch 3, Train loss: 0.1365, Val loss: 0.4773, Val accy: 84.56%



Predicting:   0%|          | 0/59 [00:00<?, ?it/s]

              precision    recall  f1-score   support

  background       0.89      0.87      0.88       997
      method       0.90      0.83      0.86       605
      result       0.71      0.89      0.79       259

    accuracy                           0.86      1861
   macro avg       0.83      0.86      0.85      1861
weighted avg       0.87      0.86      0.86      1861



In [ ]:
# #'cleaned_text
from bert_sklearn import BertClassifier
model2 = BertClassifier('bert-base-uncased',
                            max_seq_length=178,
                            epochs=3,
                            gradient_accumulation_steps=4,
                            learning_rate=3e-5,
                            train_batch_size=32,
                            eval_batch_size=32)
                            #label_list=label_list)

print(model2)

# finetune model on train data
model2.fit(X_train, y_train)

# 导入所需的库
from sklearn.metrics import classification_report

# 获取模型在测试集上的预测结果
y_preds = model2.predict(X_test)

# 打印模型在测试集上的分类报告
print(classification_report(y_test, y_preds))

Building sklearn text classifier...
BertClassifier(eval_batch_size=32, gradient_accumulation_steps=4,
               learning_rate=3e-05, max_seq_length=178)
Loading bert-base-uncased model...
Defaulting to linear classifier/regressor
Loading Pytorch checkpoint

train data size: 6588, validation data size: 732



/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


Training  :   0%|          | 0/824 [00:00<?, ?it/s]

Validating:   0%|          | 0/23 [00:00<?, ?it/s]


Epoch 1, Train loss: 0.5425, Val loss: 0.4687, Val accy: 79.64%



...

Training  :   0%|          | 0/824 [00:00<?, ?it/s]

Validating:   0%|          | 0/23 [00:00<?, ?it/s]


Epoch 3, Train loss: 0.1408, Val loss: 0.4693, Val accy: 84.97%



Predicting:   0%|          | 0/59 [00:00<?, ?it/s]

              precision    recall  f1-score   support

  background       0.89      0.86      0.88       997
      method       0.89      0.83      0.86       605
      result       0.71      0.90      0.79       259

    accuracy                           0.86      1861
   macro avg       0.83      0.86      0.84      1861
weighted avg       0.86      0.86      0.86      1861



In [ ]:
# token
from bert_sklearn import BertClassifier
model2 = BertClassifier('bert-base-uncased',
                            max_seq_length=178,
                            epochs=3,
                            gradient_accumulation_steps=4,
                            learning_rate=3e-5,
                            train_batch_size=32,
                            eval_batch_size=32)
                            #label_list=label_list)

print(model2)

# finetune model on train data
model2.fit(X_train, y_train)

# 导入所需的库
from sklearn.metrics import classification_report

# 获取模型在测试集上的预测结果
y_preds = model2.predict(X_test)

# 打印模型在测试集上的分类报告
print(classification_report(y_test, y_preds))

Building sklearn text classifier...
BertClassifier(eval_batch_size=32, gradient_accumulation_steps=4,
               learning_rate=3e-05, max_seq_length=178)
Loading bert-base-uncased model...
Defaulting to linear classifier/regressor
Loading Pytorch checkpoint

train data size: 6588, validation data size: 732



/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


Training  :   0%|          | 0/824 [00:00<?, ?it/s]

Validating:   0%|          | 0/23 [00:00<?, ?it/s]


Epoch 1, Train loss: 0.6802, Val loss: 0.5231, Val accy: 77.73%



...

Training  :   0%|          | 0/824 [00:00<?, ?it/s]

Validating:   0%|          | 0/23 [00:00<?, ?it/s]


Epoch 3, Train loss: 0.2206, Val loss: 0.4628, Val accy: 82.38%



Predicting:   0%|          | 0/59 [00:00<?, ?it/s]

              precision    recall  f1-score   support

  background       0.89      0.85      0.87       997
      method       0.86      0.83      0.84       605
      result       0.69      0.86      0.77       259

    accuracy                           0.85      1861
   macro avg       0.81      0.85      0.83      1861
weighted avg       0.85      0.85      0.85      1861



In [ ]:
from bert_sklearn import BertClassifier
model3 = BertClassifier('bert-large-uncased',
                            max_seq_length=178,
                            epochs=3,
                            gradient_accumulation_steps=4,
                            learning_rate=3e-5,
                            train_batch_size=32,
                            eval_batch_size=32)
                            #label_list=label_list)

print(model3)

# finetune model on train data
model3.fit(X_train, y_train)


# finetune model on train data
model3.fit(X_train, y_train)

# 导入所需的库
from sklearn.metrics import classification_report

# 获取模型在测试集上的预测结果
y_preds = model3.predict(X_test)

# 打印模型在测试集上的分类报告
print(classification_report(y_test, y_preds))

Building sklearn text classifier...
BertClassifier(bert_model='bert-large-uncased', eval_batch_size=32,
               gradient_accumulation_steps=4, learning_rate=3e-05,
               max_seq_length=178)


100%|██████████| 231508/231508 [00:00<00:00, 2633139.44B/s]


Loading bert-large-uncased model...


100%|██████████| 434/434 [00:00<00:00, 1423243.11B/s]


Defaulting to linear classifier/regressor
Loading Pytorch checkpoint

train data size: 6588, validation data size: 732



...

Training  :   0%|          | 0/824 [00:00<?, ?it/s]

Validating:   0%|          | 0/23 [00:00<?, ?it/s]


Epoch 1, Train loss: 0.8450, Val loss: 0.9341, Val accy: 58.61%



Training  :   0%|          | 0/824 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
from bert_sklearn import BertClassifier
# model3 = BertClassifier('scibert-scivocab-uncased',
#                             max_seq_length=128,
#                             epochs=3,
#                             gradient_accumulation_steps=4,
#                             learning_rate=3e-5,
#                             train_batch_size=32,
#                             eval_batch_size=32)
#                             #label_list=label_list)
model3 = BertClassifier('scibert-scivocab-uncased',
                       label_list=None,
                       num_mlp_hiddens=500,
                       num_mlp_layers=0,
                       epochs=3,
                       max_seq_length=128,
                       train_batch_size=32,
                       eval_batch_size=8,
                       learning_rate=2e-5,
                       warmup_proportion=0.1,
                       gradient_accumulation_steps=1,
                       fp16=False,
                       loss_scale=0,
                       local_rank=-1,
                       use_cuda=True,
                       random_state=42,
                       validation_fraction=0.1,
                       logfile='bert_sklearn.log')
print(model3)


# finetune model on train data
model3.fit(X_train, y_train)

# 导入所需的库
from sklearn.metrics import classification_report

# 获取模型在测试集上的预测结果
y_preds = model3.predict(X_test)

# 打印模型在测试集上的分类报告
print(classification_report(y_test, y_preds))

Building sklearn text classifier...
BertClassifier(bert_model='scibert-scivocab-uncased')


100%|██████████| 410593280/410593280 [00:24<00:00, 17021290.04B/s]


Loading scibert-scivocab-uncased model...


100%|██████████| 410593280/410593280 [00:09<00:00, 44026018.36B/s]


Defaulting to linear classifier/regressor
Loading Pytorch checkpoint

train data size: 6588, validation data size: 732



...

Training  :   0%|          | 0/206 [00:00<?, ?it/s]

Validating:   0%|          | 0/92 [00:00<?, ?it/s]


Epoch 3, Train loss: 0.1178, Val loss: 0.3903, Val accy: 87.16%



Predicting:   0%|          | 0/233 [00:00<?, ?it/s]

              precision    recall  f1-score   support

  background       0.89      0.87      0.88       997
      method       0.88      0.84      0.86       605
      result       0.75      0.88      0.81       259

    accuracy                           0.86      1861
   macro avg       0.84      0.87      0.85      1861
weighted avg       0.87      0.86      0.86      1861



In [ ]:
# token
from bert_sklearn import BertClassifier
model4 = BertClassifier('bert-base-cased',
                            max_seq_length=178,
                            epochs=3,
                            gradient_accumulation_steps=4,
                            learning_rate=3e-5,
                            train_batch_size=32,
                            eval_batch_size=32)
                            #label_list=label_list)

print(model4)

# finetune model on train data
model4.fit(X_train, y_train)

# 导入所需的库
from sklearn.metrics import classification_report

# 获取模型在测试集上的预测结果
y_preds = model4.predict(X_test)

# 打印模型在测试集上的分类报告
print(classification_report(y_test, y_preds))

Building sklearn text classifier...
BertClassifier(bert_model='bert-base-cased', eval_batch_size=32,
               gradient_accumulation_steps=4, learning_rate=3e-05,
               max_seq_length=178)


100%|██████████| 213450/213450 [00:00<00:00, 2542944.68B/s]


Loading bert-base-cased model...


100%|██████████| 433/433 [00:00<00:00, 1092739.85B/s]


Defaulting to linear classifier/regressor
Loading Pytorch checkpoint

train data size: 6588, validation data size: 732



...

Training  :   0%|          | 0/824 [00:00<?, ?it/s]

Validating:   0%|          | 0/23 [00:00<?, ?it/s]


Epoch 3, Train loss: 0.1253, Val loss: 0.4235, Val accy: 86.20%



Predicting:   0%|          | 0/59 [00:00<?, ?it/s]

              precision    recall  f1-score   support

  background       0.88      0.88      0.88       997
      method       0.88      0.82      0.85       605
      result       0.74      0.88      0.80       259

    accuracy                           0.86      1861
   macro avg       0.83      0.86      0.84      1861
weighted avg       0.86      0.86      0.86      1861



In [ ]:
# token
from bert_sklearn import BertClassifier
model5 = BertClassifier('scibert-basevocab-uncased',
                            max_seq_length=178,
                            epochs=3,
                            gradient_accumulation_steps=4,
                            learning_rate=3e-5,
                            train_batch_size=32,
                            eval_batch_size=32)
                            #label_list=label_list)

print(model5)

# finetune model on train data
model5.fit(X_train, y_train)

# 导入所需的库
from sklearn.metrics import classification_report

# 获取模型在测试集上的预测结果
y_preds = model5.predict(X_test)

# 打印模型在测试集上的分类报告
print(classification_report(y_test, y_preds))


Building sklearn text classifier...
BertClassifier(bert_model='scibert-basevocab-uncased', eval_batch_size=32,
               gradient_accumulation_steps=4, learning_rate=3e-05,
               max_seq_length=178)


100%|██████████| 407500800/407500800 [00:22<00:00, 18511193.07B/s]


Loading scibert-basevocab-uncased model...


100%|██████████| 407500800/407500800 [00:08<00:00, 50054410.74B/s]


Defaulting to linear classifier/regressor
Loading Pytorch checkpoint

train data size: 6588, validation data size: 732



...

Training  :   0%|          | 0/824 [00:00<?, ?it/s]

Validating:   0%|          | 0/23 [00:00<?, ?it/s]


Epoch 3, Train loss: 0.1537, Val loss: 0.4551, Val accy: 85.38%



Predicting:   0%|          | 0/59 [00:00<?, ?it/s]

              precision    recall  f1-score   support

  background       0.90      0.88      0.89       997
      method       0.89      0.83      0.86       605
      result       0.72      0.89      0.80       259

    accuracy                           0.87      1861
   macro avg       0.84      0.87      0.85      1861
weighted avg       0.87      0.87      0.87      1861

